# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [251]:
# import the requests library (1 line)
import requests
# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url='https://country-leaders.onrender.com'
# assign the /status endpoint to another variable called status_url (1 line)
status_url='/status'
# query the /status endpoint using the get() method and store it in the req variable (1 line)
req=requests.get((f"{root_url}/{status_url}"))
# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code==200: 
    print (req.text) 
else: 
    print (req.status_code)



"Alive"


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [252]:
# Set the countries_url variable (1 line)
countries_url="https://country-leaders.onrender.com/countries"
# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req=requests.get((f"{countries_url}"))
# Get the JSON content and store it in the countries variable (1 line)
countries=req.json()
# display the request's status code and the countries variable (1 line)
req.status_code
countries


{'message': 'The cookie is missing'}

### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [253]:
# Set the cookie_url variable (1 line)
cookie_url='https://country-leaders.onrender.com/cookie'
# Query the enpoint, set the cookies variable and display it (2 lines)
cookies=requests.get(f"{cookie_url}")
cookies=cookies.cookies


Try to query the countries endpoint using the cookie, save the output and print it.

In [254]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries=requests.get((f"{countries_url}"),cookies=cookies)
# display the countries variable (1 line)
countries

<Response [200]>

Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [255]:
# Set the leaders_url variable (1 line)
leaders_url='https://country-leaders.onrender.com/leaders'
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders=requests.get((f"{leaders_url}"),cookies=cookies)
# display the leaders variable (1 line)
leaders

<Response [404]>

It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [256]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
countries=countries.json() #countries is a response object, not a dictionary. This method changes it to a dict so params can use it
params={"country":countries[0]}
# assign the output to the leaders variable (1 line)
leaders=requests.get((f"{leaders_url}"),params=params,cookies=cookies)
# display the leaders variable (1 line)
leaders


<Response [200]>

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [257]:
# 4 lines
leaders_per_country={}
for country in countries:
    leaders_per_country[country]=requests.get((f"{leaders_url}"),params={"country": country},cookies=cookies).json()
print(leaders_per_country)

{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

In [258]:
# or 1 line


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [259]:
# < 15 lines
from requests import get


def get_leaders():
    #defining the urls:
    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leaders_url = f"{root_url}/leaders"
    #getting the cookie
    cookies=requests.get(f"{cookie_url}")
    cookies=cookies.cookies
    #getting countries
    countries=requests.get(f"{countries_url}", cookies=cookies)
    countries=countries.json()
    #creating an empty dictionary for saving leaders per country
    leaders_per_country={}
    #looping over countries to save their leaders 
    for country in countries:
        leaders_per_country[country]=requests.get((f"{leaders_url}"),params={"country": country},cookies=cookies).json()
    return(leaders_per_country)


Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [260]:
get_leaders()


{'be': [{'id': 'Q12978',
   'first_name': 'Guy',
   'last_name': 'Verhofstadt',
   'birth_date': '1953-04-11',
   'death_date': None,
   'place_of_birth': 'Dendermonde',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt',
   'start_mandate': '1999-07-12',
   'end_mandate': '2008-03-20'},
  {'id': 'Q12981',
   'first_name': 'Yves',
   'last_name': 'Leterme',
   'birth_date': '1960-10-06',
   'death_date': None,
   'place_of_birth': 'Wervik',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme',
   'start_mandate': '2009-11-25',
   'end_mandate': '2011-12-06'},
  {'id': 'Q12983',
   'first_name': 'Herman',
   'last_name': 'None',
   'birth_date': '1947-10-31',
   'death_date': None,
   'place_of_birth': 'Etterbeek',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy',
   'start_mandate': '2008-12-30',
   'end_mandate': '2009-11-25'},
  {'id': 'Q14989',
   'first_name': 'Léon',
   'last_name': 'Delacroix',
   'birth_date': '1867-12-27',
   'd

## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [261]:
# 3 lines

wiki_url= leaders[0]["wikipedia_url"]
html_text = requests.get(wiki_url, headers=headers).text
#displaying:
html_text

TypeError: 'Response' object is not subscriptable

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [263]:
# 3 lines
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_text, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="nl">
 <head>
  <meta charset="utf-8"/>
  <title>
   Guy Verhofstadt - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [264]:
# 2 lines
paragraphs=soup.find_all("p")
[print(paragraph) for paragraph in paragraphs]

<p><b>Guy Maurice Marie-Louise Verhofstadt</b><sup class="reference" id="cite_ref-5"><a href="#cite_note-5"><span class="cite-bracket">[</span>5<span class="cite-bracket">]</span></a></sup> (<span class="ext-phonos"><span class="noexcerpt ext-phonos-PhonosButton oo-ui-widget oo-ui-widget-enabled oo-ui-buttonElement oo-ui-buttonElement-frameless oo-ui-iconElement oo-ui-labelElement oo-ui-buttonWidget" data-nosnippet="" data-ooui='{"_":"mw.Phonos.PhonosButton","href":"\/\/upload.wikimedia.org\/wikipedia\/commons\/transcoded\/d\/dc\/Nl-be_guy_verhofstadt.ogg\/Nl-be_guy_verhofstadt.ogg.mp3","rel":["nofollow"],"framed":false,"icon":"volumeUp","label":{"html":"uitspraak"},"data":{"ipa":"","text":"","lang":"nl","wikibase":"","file":"Nl-be guy verhofstadt.ogg"},"classes":["noexcerpt","ext-phonos-PhonosButton"]}' id="ooui-php-1"><a aria-label="Geluid afspelen" class="oo-ui-buttonElement-button" href="//upload.wikimedia.org/wikipedia/commons/transcoded/d/dc/Nl-be_guy_verhofstadt.ogg/Nl-be_guy_ve

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [265]:
# <10 lines
for paragraph in soup.find_all("p"):
    if paragraph.find("b")==paragraph.contents[0]: #checks if our <p> element starts with bold <b> text.
        first_paragraph=paragraph.get_text()
        break

At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [266]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph
def get_first_paragraph(wikipedia_url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) ..."}
    print(wikipedia_url) # keep this for the rest of the notebook
    html_text = requests.get(wikipedia_url, headers=headers).text
    soup = BeautifulSoup(html_text, "html.parser")
    for paragraph in soup.find_all("p"):
        if paragraph.find("b")==paragraph.contents[0]: #checks if our <p> element starts with bold <b> text.
            first_paragraph=paragraph.get_text()
            return first_paragraph

In [267]:
# Test: 3 lines
first_paragraph=get_first_paragraph("https://pl.wikipedia.org/wiki/Abraham_Lincoln")
print(first_paragraph)

https://pl.wikipedia.org/wiki/Abraham_Lincoln
Abraham Lincoln (ur. 12 lutego 1809 pod Hodgenville, zm. 15 kwietnia 1865 w Waszyngtonie) – amerykański polityk, mąż stanu, 16. prezydent Stanów Zjednoczonych. 



### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [268]:
# 10 lines
def clean_first_paragraph(first_paragraph):
    first_paragraph #a
    patterns=[r"\[\d+\]", #references like [42]
            r"<span.*?>.*?</span>", #<span> blocks - phonetic buttons, styling
            r"\([^\)]*\)", #phonetic pronunciations
            r"[\u2460-\u24FF]", # Extra unicode symbols like ⓘ etc
            r"<.*?>" #HTML tags (if there are any)
            ]#this is a list that contains all the regex patterns to remove from the text for cleaning

    repl=""
    
    cleaned=first_paragraph
    for pattern in patterns: #made a loop here to see the effects of each applied regex pattern for testing
        cleaned= re.sub(pattern,"",cleaned,flags=re.DOTALL) #the flag makes it detect things through new lines (/n)
        print(cleaned)
#testing
test=clean_first_paragraph(first_paragraph)

Abraham Lincoln (ur. 12 lutego 1809 pod Hodgenville, zm. 15 kwietnia 1865 w Waszyngtonie) – amerykański polityk, mąż stanu, 16. prezydent Stanów Zjednoczonych. 

Abraham Lincoln (ur. 12 lutego 1809 pod Hodgenville, zm. 15 kwietnia 1865 w Waszyngtonie) – amerykański polityk, mąż stanu, 16. prezydent Stanów Zjednoczonych. 

Abraham Lincoln  – amerykański polityk, mąż stanu, 16. prezydent Stanów Zjednoczonych. 

Abraham Lincoln  – amerykański polityk, mąż stanu, 16. prezydent Stanów Zjednoczonych. 

Abraham Lincoln  – amerykański polityk, mąż stanu, 16. prezydent Stanów Zjednoczonych. 



Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [269]:
# 10 lines
def clean_first_paragraph(first_paragraph):
    first_paragraph #a
    patterns=[r"\[\d+\]", #references like [42]
            r"<span.*?>.*?</span>", #<span> blocks - phonetic buttons, styling
            r"\([^\)]*\)", #phonetic pronunciations
            r"[\u2460-\u24FF]", # Extra unicode symbols like ⓘ etc
            r"<.*?>" #HTML tags (if there are any)
            ]#this is a list that contains all the regex patterns to remove from the text for cleaning

    repl=""
    
    cleaned=first_paragraph
    for pattern in patterns: #made a loop here to see the effects of each applied regex pattern for testing
        cleaned= re.sub(pattern,"",cleaned,flags=re.DOTALL) #the flag makes it detect things through new lines (/n)
    print(cleaned)



def get_first_paragraph(wikipedia_url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) ..."}
    print(wikipedia_url) # keep this for the rest of the notebook
    html_text = requests.get(wikipedia_url, headers=headers).text
    soup = BeautifulSoup(html_text, "html.parser")
    for paragraph in soup.find_all("p"):
        if paragraph.find("b")==paragraph.contents[0]: #checks if our <p> element starts with bold <b> text.
            first_paragraph=paragraph.get_text()
            return clean_first_paragraph(first_paragraph)

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [270]:
# < 20 lines
def get_leaders():
    #defining the urls:
    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leaders_url = f"{root_url}/leaders"
    #getting the cookie
    cookies=requests.get(f"{cookie_url}")
    cookies=cookies.cookies
    #getting countries
    countries=requests.get(f"{countries_url}", cookies=cookies)
    countries=countries.json()
    #creating an empty dictionary for saving leaders per country
    leaders_per_country={}
    #looping over countries to save their leaders 
    for country in countries:
        leaders_per_country[country]=requests.get((f"{leaders_url}"),params={"country": country},cookies=cookies).json()
        for leader in leaders:
            wiki_url=leader.get("wikipedia_url")
            if wiki_url: #only if URL exists
                leader["first_paragraph"]=get_first_paragraph(wiki_url)
    return(leaders_per_country)




In [271]:
# Check the output of your function (2 lines)
get_leaders()

AttributeError: 'bytes' object has no attribute 'get'

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [274]:
# < 25 lines
def get_cookie(): 
    cookie=requests.get(f"{root_url}/cookie")
    cookie.raise_for_status()
    return {"user_cookie": cookie.cookies.get("user_cookie")}
    

def get_leaders():
    #defining the urls:
    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leaders_url = f"{root_url}/leaders"
    
    cookies=get_cookie()
    #getting countries
    try:
        countries=requests.get(f"{countries_url}", cookies=cookies)
        countries.raise_for_status()
        countries=countries.json()
        for country in countries:
            retry_attempt=0
            while retry_attempt<3: #this block will make the function retry 3 times when encountered an error before going further
                try:
                    leaders = requests.get(f"{leaders_url}", params={"country": country}, cookies=cookies)
                    leaders.raise_for_status()
                    leaders = leaders.json()
                    if isinstance(leaders, dict) and "message" in leaders and "expired" in leaders["message"].lower():
                        # Cookie expired, refresh
                        cookies = get_cookie()
                        retries += 1
                        continue
                    for leader in leaders:
                        wiki_url=leader.get("wikipedia_url")
                        if wiki_url: #only if URL exists
                            leader["first_paragraph"]=get_first_paragraph(wiki_url)

                    leaders_per_country[country] = leaders
                    break
                except (requests.RequestException, ValueError) as e:
                    print(f"Error fetching leaders for {country}: {e}")
                    leaders_per_country[country] = []
                    break

    except (requests.RequestException, ValueError) as e:
        print(f"Failed to fetch countries or cookie: {e}")
    
    return(leaders_per_country)


Check the output of your function again.

In [275]:
# Check the output of your function (1 line)
get_leaders()

https://nl.wikipedia.org/wiki/Guy_Verhofstadt
Guy Maurice Marie-Louise Verhofstadt   is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten . Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Hij beëindigde zijn actieve politieke carrière in het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa  was.

https://nl.wikipedia.org/wiki/Yves_Leterme
Yves Camille Désiré Leterme  is een voormalig Belgisch politicus. Hij was onder meer Vlaams minister-president en premier van België. Hij was lang boegbeeld van de CD&V en boegbeeld van het toenmalige kartel CD&V/N-VA.

https://nl.wikipedia.org/wiki/Herman_Van_Rompuy
Herman Achille graaf Van Rompuy  is een Belgische politicus van de christendemocratische partij CD&V. Hij was tussen 30 december 2008 en 25 november 2009 de premier van België en van 1 januari 2010 tot en met 30 november 2014 voorzitter van de Europese Raad, in de media

KeyboardInterrupt: 

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [276]:
# < 20 lines

        
def get_first_paragraph(wikipedia_url,session):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) ..."}
    print(wikipedia_url) # keep this for the rest of the notebook
    html_text = session.get(wikipedia_url, headers=headers).text
    soup = BeautifulSoup(html_text, "html.parser")
    for paragraph in soup.find_all("p"):
        if paragraph.find("b")==paragraph.contents[0]: #checks if our <p> element starts with bold <b> text.
            first_paragraph=paragraph.get_text()
            return clean_first_paragraph(first_paragraph)

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [278]:
# <25 lines


def get_leaders():
    session=requests.Session()
    #defining the urls:
    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leaders_url = f"{root_url}/leaders"
    
    cookies=get_cookie()
    #getting countries
    try:
        countries=requests.get(f"{countries_url}", cookies=cookies)
        countries.raise_for_status()
        countries=countries.json()
        for country in countries:
            retry_attempt=0
            while retry_attempt<3: #this block will make the function retry 3 times when encountered an error before going further
                try:
                    leaders = requests.get(f"{leaders_url}", params={"country": country}, cookies=cookies)
                    leaders.raise_for_status()
                    leaders = leaders.json()
                    if isinstance(leaders, dict) and "message" in leaders and "expired" in leaders["message"].lower():
                        # Cookie expired, refresh
                        cookies = get_cookie()
                        retries += 1
                        continue
                    for leader in leaders:
                        wiki_url=leader.get("wikipedia_url")
                        if wiki_url: #only if URL exists
                            leader["first_paragraph"]=get_first_paragraph(wiki_url,session)

                    leaders_per_country[country] = leaders
                    break
                except (requests.RequestException, ValueError) as e:
                    print(f"Error fetching leaders for {country}: {e}")
                    leaders_per_country[country] = []
                    break

    except (requests.RequestException, ValueError) as e:
        print(f"Failed to fetch countries or cookie: {e}")
    
    return(leaders_per_country)


Test your new functions.



In [279]:
get_leaders()

https://nl.wikipedia.org/wiki/Guy_Verhofstadt
Guy Maurice Marie-Louise Verhofstadt   is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten . Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Hij beëindigde zijn actieve politieke carrière in het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa  was.

https://nl.wikipedia.org/wiki/Yves_Leterme
Yves Camille Désiré Leterme  is een voormalig Belgisch politicus. Hij was onder meer Vlaams minister-president en premier van België. Hij was lang boegbeeld van de CD&V en boegbeeld van het toenmalige kartel CD&V/N-VA.

https://nl.wikipedia.org/wiki/Herman_Van_Rompuy
Herman Achille graaf Van Rompuy  is een Belgische politicus van de christendemocratische partij CD&V. Hij was tussen 30 december 2008 en 25 november 2009 de premier van België en van 1 januari 2010 tot en met 30 november 2014 voorzitter van de Europese Raad, in de media

KeyboardInterrupt: 

## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [283]:

# 3 lines
import json
with open("leaders.json", "w") as f:
    json.dump(leaders_per_country,f)



    
    

Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [284]:
# 3 lines
with open("leaders.json") as f:
    leaders_per_country_copy= json.load(f)
print(leaders_per_country_copy==leaders_per_country) # reads the copy and compares it with the original dictionary. Prints true if they match.


True


Make a function `save(leaders_per_country)` to call this code easily.

In [285]:
# 3 lines
def save(leaders_per_country):
    with open('leaders.json', 'w') as f:
        json.dump(leaders_per_country, f)

In [286]:
# Call the function (1 line)
save(leaders_per_country)

## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!